In [1]:
# demo.py
# 사용법: python demo.py
# 필요: pip install pyyaml

import re
import math
import random
import yaml
from copy import deepcopy

DEFAULT_TAU = "T3"  # yaml에 default_timing 없을 때 폴백

def _apply_defaults(params, action_space, globals_):
    out = {}
    out.update(globals_ or {})
    for k, spec in (action_space or {}).items():
        if isinstance(spec, dict) and "default" in spec and k not in params:
            out[k] = spec["default"]
    out.update(params or {})
    return out

def _render_template(tpl, ctx):
    # {{var|default:VAL}}
    def repl_default(m):
        var, val = m.group(1), m.group(2)
        return str(_get(ctx, var, default=val))
    tpl = re.sub(r"\{\{([\w\.]+)\|default:([^}]+)\}\}", repl_default, tpl)

    # {{#cond}}...{{/cond}}
    def block_sub(match):
        key = match.group(1)
        body = match.group(2)
        v = _get(ctx, key)
        return body if v else ""
    tpl = re.sub(r"\{\{#(\w+)\}\}([\s\S]*?)\{\{/\1\}\}", block_sub, tpl)

    # {{var}}
    def repl_var(m):
        key = m.group(1)
        v = _get(ctx, key)
        return "" if v is None else str(v)
    tpl = re.sub(r"\{\{([\w\.]+)\}\}", repl_var, tpl)

    # 공백 정리
    return re.sub(r"[ \t]+", " ", " ".join(tpl.split())).strip()

def _get(ctx, dotted, default=None):
    if "." not in dotted:
        return ctx.get(dotted, default)
    cur = ctx
    for part in dotted.split("."):
        if not isinstance(cur, dict) or part not in cur:
            return default
        cur = cur[part]
    return cur

class NmapTool:
    def __init__(self, tools_yaml_path="tools.yaml"):
        with open(tools_yaml_path, "r", encoding="utf-8") as f:
            self.spec = yaml.safe_load(f)
        self.actions = {a["name"]: a for a in self.spec.get("actions", [])}
        self.globals = self.spec.get("globals", {}) or {}

    def render(self, action_name, **params):
        if action_name not in self.actions:
            raise KeyError(f"unknown action: {action_name}")
        action = deepcopy(self.actions[action_name])
        cmd_tpl = action["command_template"]
        ctx = _apply_defaults(params, action.get("action_space"), self.globals)
        # timing 기본값 폴백
        ctx.setdefault("timing", self.globals.get("default_timing", DEFAULT_TAU))
        # 렌더링
        return _render_template(cmd_tpl, {"globals": self.globals, **ctx})

def main():
    tool = NmapTool("tools.yaml")

    cases = [
        ("host_discovery", dict(targets="10.0.0.0/24", save=True, out_prefix="scan")),
        ("tcp_syn_top", dict(targets="10.0.0.5", top_n=1000, no_ping=True)),
        ("tcp_connect_list", dict(targets="web.example.com", ports="80,443", timing="T4")),
        ("udp_top", dict(targets="10.0.0.5", top_n=200)),
        ("service_version", dict(targets="10.0.0.5", ports="22,80,443")),
        ("aggressive_profile", dict(targets="10.0.0.5", top_n=200, no_ping=True)),
        ("banner_grab", dict(targets="10.0.0.5", ports="80")),
        ("timing_tune", dict(targets="10.0.0.5", top_n=100, max_rate=5000, retries=2, host_timeout=60)),
    ]

    for name, params in cases:
        cmd = tool.render(name, **params)
        print(f"[{name}]")
        print(cmd)
        print("-" * 80)

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'tools.yaml'